In [1]:
import os
import math
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from hydroeval import nse, kge, pbias

from fancyimpute import IterativeImputer as MICE
from fancyimpute import NuclearNormMinimization, SoftImpute

import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout, BatchNormalization, ConvLSTM2D, Bidirectional, Flatten
from keras.callbacks import ReduceLROnPlateau
from keras import backend

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
def get_grdc(grdc_no):
    data_date_month = []
    data_date_common = []
    data_value = []
    f = open("final_asia/" + grdc_no + "_Q_Day.Cmd.txt", "rb")
    for i in f:
        i = str(i)[2:-5]
        if "#" in i:
            continue
        if "YYYY" in i:
            continue
        temp = [i.strip() for i in i.split(";")]
        date_temp = temp[0].split("-")
        date_temp = [i.strip() for i in temp[0].split("-")]
        data_date_month.append(int(date_temp[1]))
        data_date_common.append(date_temp[0] + "-" + date_temp[1] + "-" + date_temp[2])
        data_value.append(round(float(temp[2]), 3))
    df_temp = pd.DataFrame(list(zip(data_date_month, data_value, data_date_common)),
                               columns =["Month", "Value", "Date"])
    df_temp = df_temp.set_index('Date')
    df_temp = df_temp[df_temp["Month"] != 0]
    df_temp.sort_index()
    df_index = df_temp.index
    df_temp.replace(-999.0, np.nan, inplace=True)
    
    return df_temp

In [3]:
def get_data(f):
    
    value = []
    common = []
    
    grdc_no = ""
    latitude = ""
    longitude = ""
    river = ""
    
    for i in f:
        
        i = str(i)[2:-5]
        
        if "GRDC-No" in i:
            grdc_no = i.split(":")[1].strip()
            continue
        elif "Latitude" in i:
            latitude = i.split(":")[1].strip()
            continue
        elif "Longitude" in i:
            longitude = i.split(":")[1].strip()
            continue
        elif "River" in i:
            river = i.split(":")[1].strip()
            river = river.replace(',', '')
            continue
        elif "YYYY" in i:
            continue
        elif "#" in i:
            continue

        temp = [i.strip() for i in i.split(";")]
        date_temp = temp[0].split("-")
        date_temp = [i.strip() for i in temp[0].split("-")]
        common.append(date_temp[0] + "-" + date_temp[1] + "-" + date_temp[2])
        value.append(round(float(temp[2]), 3))
        
    return value, common, grdc_no, latitude, longitude, river

In [4]:
def merged(x, x_ncdc):
    
    x = x.set_index('Date')
    x.sort_index()
    x_ncdc = x_ncdc.set_index('DATE')
    x_ncdc.sort_index()
    
    x_first = x.index[0] if x.index[0] > x_ncdc.index[0] else x_ncdc.index[0]
    x_last = x.index[-1] if x.index[-1] < x_ncdc.index[-1] else x_ncdc.index[-1]
    merged = pd.concat([x, x_ncdc], axis=1, sort=True)
    to_delete = []
    for i, r in merged.iterrows():
        if i < x_first or i > x_last:
            to_delete.append(i)
            
    merged = merged.drop(to_delete)
    
    return merged


def merged2(x, x_ncdc):
    x = x.set_index('Date')
    x.sort_index()
    x_ncdc = x_ncdc.set_index('DATE')
    x_ncdc.sort_index()
    merged = pd.concat([x, x_ncdc], axis=1, sort=True)
    return merged

In [ ]:
all_data = {}

for filename in os.listdir("final_asia/"):

    f = open("final_asia/" + filename, "rb")
    f2 = pd.read_csv("NCDC/" + filename[:7] + ".csv")

    value, common, grdc_no, latitude, longitude, river = get_data(f)
    f.close()

    df_temp = pd.DataFrame(list(zip(value, common)), columns =["Value", "Date"])
    df_temp = merged(df_temp, f2)
    df2 = df_temp["Value"]
    df2 = df2.replace(-999.0, np.nan)
    df2 = df2.fillna(df2.mean())
    first_index = df_temp["LONGITUDE"].first_valid_index()
    temp_dict = {
        "GRDC_latitude": latitude,
        "GRDC_longitude": longitude,
        "NCDC_latitude": df_temp["LATITUDE"][first_index],
        "NCDC_longitude": df_temp["LONGITUDE"][first_index],
        "Elevation": df_temp["ELEVATION"][first_index],
        "River": river
    }
    cols = list(df_temp.columns)
    cols_to_keep = []
    for i in cols:
        if len(i) == 4 and i != "NAME" and i != "SNWD":
            cols_to_keep.append(i)
            cols.remove(i)
    df_temp.drop(columns=cols, inplace=True)
    df_temp.sort_index()
    df_index = df_temp.index
    soft_impute = SoftImpute(max_iters=300, verbose=False)
    df_temp = soft_impute.fit_transform(df_temp)
    df_temp = pd.DataFrame(data=df_temp, columns=cols_to_keep)
    if "PRCP" in cols_to_keep:
        df_temp[df_temp["PRCP"] < 0] = 0
    df_temp["Date"] = df_index
    df_temp = df_temp.set_index(["Date"])
    df_temp["t-1"] = df2.shift(1)
    df_temp = df_temp.replace(np.nan, 0)
    temp_dict["data_x"] = df_temp
    temp_dict["data_y"] = df2
    all_data[grdc_no] = temp_dict

In [29]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		end_ix = i + n_steps
		if end_ix > len(sequences)-1:
			break
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
new_data_x2 = all_data["2998400"]["data_x"]
new_data_y = all_data["2998400"]["data_y"]
new_data_y2 = np.asarray(new_data_y).reshape((-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
new_data_x2 = scaler.fit_transform(new_data_x)
new_data_y2 = scaler.fit_transform(new_data_y2)
train_size = round(0.8*new_data_x2.shape[0])
X_train = new_data_x2[:train_size]
y_train = new_data_y2[:train_size]
X_test = new_data_x2[train_size:]
y_test = new_data_y2[train_size:]
X_train2 = np.asarray(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
y_train2 = np.asarray(y_train)
X_test2 = np.asarray(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
y_test2 = np.asarray(y_test)

model = Sequential()
model.add(LSTM(8, input_shape=[X_train2.shape[1], X_train2.shape[2]]))
model.add(Dense(1))
model.summary()
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
history = model.fit(X_train2, y_train2 ,epochs = 10)

In [ ]:
model_new = Sequential()
model_new.add(LSTM(64, input_shape=[X_train2.shape[1], X_train2.shape[2]]))
model_new.add(Dense(1))
model_new.set_weights(model.get_weights())
model_new.summary()
model_new.compile(loss='mean_absolute_error', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
tails = 5000
heads = 90
for i in all_data_test:
    if i != "2969200":
        continue
    new_data_x = all_data_test[i]["data_x"]
    new_data_y = all_data_test[i]["data_y"]
    new_data_old = np.asarray(all_data[i]["data_y"].tail(tails))
    new_data_x = new_data_x.loc[all_data[i]["data_x"].index[all_data[i]["data_x"].shape[0]-1]:]
    new_data_y = new_data_y.loc[all_data[i]["data_x"].index[all_data[i]["data_x"].shape[0]-1]:]
    x_axis = list(all_data[i]["data_y"].tail(tails).index) + list(new_data_x.index)
    temp3 = len(x_axis)
    temp2 = round(temp3/8)
    x_labels = []
    for j in range(0, temp3, temp2):
        x_labels.append(x_axis[j])
    if len(x_labels) < 9:
        x_labels.append(x_axis[-1])

    new_data_y2 = np.asarray(new_data_y).reshape((-1, 1))
    initial = 0
    new_data_x = np.asarray(new_data_x)
    predictions_all = []
    for i in new_data_x:
        temp = np.append(i, initial)
        temp = model_new.predict(temp.reshape(1,5,1))
        initial = temp[0][0]
        predictions_all.append(initial)
        
    results_train = predictions_all
    plt.figure(figsize=(20,5))
    plt.plot(range(len(new_data_old)), new_data_old, label="Actual")
    plt.plot(range(len(new_data_old), len(new_data_old) + len(results_train)), results_train, label="")
    plt.xticks(range(0,temp3, temp2), x_labels)
    plt.legend(['Actual','Predicted'])
    plt.xlabel("Date")
    plt.ylabel("Water Runoff")
    plt.show()